In [1]:
from triagerx.model.module_factory import ModelFactory
from loguru import logger
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from triagerx.dataset.text_processor import TextProcessor
from tqdm import tqdm

tqdm.pandas()

In [2]:
torch.manual_seed(42)

In [5]:
target_components = ["comp:vm", "comp:jvmti", "comp:jclextensions", "comp:test", "comp:build", "comp:gc"]
target_components = sorted(target_components)

In [6]:
# # Define active user map
# vm_users = [
#     "pshipton",
#     "keithc-ca",
#     "gacholio",
#     "tajila",
#     "babsingh",
#     "JasonFengJ9",
#     "fengxue-IS",
#     "hangshao0",
#     "theresa.t.mammarella",
#     "ChengJin01",
#     "singh264",
#     "thallium",
#     "ThanHenderson",
# ]
# jvmti_users = ["gacholio", "tajila", "babsingh", "fengxue-IS"]
# jclextensions_users = ["JasonFengJ9", "pshipton", "keithc-ca"]
# test_users = ["LongyuZhang", "annaibm", "sophiaxu0424", "KapilPowar", "llxia"]
# build_users = ["adambrousseau", "mahdipub"]
# gc_users = ["dmitripivkine", "amicic", "kangyining", "LinHu2016"]

# # Putting them in dictionaries
# components = {
#     "comp:vm": vm_users,
#     "comp:jvmti": jvmti_users,
#     "comp:jclextensions": jclextensions_users,
#     "comp:test": test_users,
#     "comp:build": build_users,
#     "comp:gc": gc_users,
# }

# # expected_users = [user for user_list in components.values() for user in user_list]
# # df = df[df["owner"].isin(expected_users)]

# # df = df[df["component"].isin(target_components)]

In [60]:
# df_train, df_test = train_test_split(df, test_size=0.1, shuffle=False)
df_train = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/data/openj9/last_contribution/openj9_train.csv")
df_test = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/data/openj9/last_contribution/openj9_test.csv")

In [61]:
expected_users = set(df_test.owner.unique())

In [62]:
sample_threshold = 20
developers = df_train["owner"].value_counts()
filtered_developers = developers.index[developers >= sample_threshold]
df_train = df_train[df_train["owner"].isin(filtered_developers)]

train_owners = set(df_train["owner"])
test_owners = set(df_test["owner"])

unwanted = list(test_owners - train_owners)

df_test = df_test[~df_test["owner"].isin(unwanted)]

logger.info(f"Training data: {len(df_train)}, Validation data: {len(df_test)}")
logger.info(f"Number of developers: {len(df_train.owner.unique())}")

logger.info(f"Train dataset size: {len(df_train)}")
logger.info(f"Test dataset size: {len(df_test)}")

2024-06-20 13:10:33.803 | INFO     | __main__:<module>:13 - Training data: 3323, Validation data: 379
2024-06-20 13:10:33.805 | INFO     | __main__:<module>:14 - Number of developers: 50
2024-06-20 13:10:33.806 | INFO     | __main__:<module>:16 - Train dataset size: 3323
2024-06-20 13:10:33.808 | INFO     | __main__:<module>:17 - Test dataset size: 379


In [65]:
# # Generate label ids
lbl2idx = dict(zip(df_train['owner'], df_train['owner_id']))
idx2lbl = dict(zip(df_train['owner_id'], df_train['owner']))

# train_owners = sorted(train_owners)

# for idx, dev in enumerate(train_owners):
#     lbl2idx[dev] = idx
#     idx2lbl[idx] = dev

# df_train["owner_id"] = df_train["owner"].apply(lambda owner: lbl2idx[owner])
# df_test["owner_id"] = df_test["owner"].apply(lambda owner: lbl2idx[owner])

In [67]:
df_train = df_train[df_train["component"].notna()]
df_test = df_test[df_test["component"].notna()]

In [68]:
comp_id2label = {}
comp_lbl2id = {}

for i, comp in enumerate(target_components):
    comp_id2label[i] = comp
    comp_lbl2id[comp] = i

In [69]:
base_transformer_models = ["microsoft/deberta-base", "roberta-base"]

In [70]:
developer_model_weights = "/work/disa_lab/projects/triagerx/models/openj9/triagerx_ensemble_u3_50_classes_last_dev_seed42_last_epoch_39.pt"
component_model_weights = "/work/disa_lab/projects/triagerx/models/openj9/component_triagerx_u3_6_classes_seed42.pt"

In [71]:
logger.debug("Modeling network...")
dev_model = ModelFactory.get_model(
    model_key="triagerx",
    output_size=len(df_train.owner_id.unique()),
    unfrozen_layers=3,
    num_classifiers=3,
    base_models=base_transformer_models,
    dropout=0.2,
    max_tokens=256,
    label_map=idx2lbl,
)

2024-06-20 13:11:00.147 | DEBUG    | __main__:<module>:1 - Modeling network...
2024-06-20 13:11:00.149 | DEBUG    | triagerx.model.module_factory:get_model:54 - Instantiating model of class: <class 'triagerx.model.triagerx_dev_model.TriagerxDevModel'>
2024-06-20 13:11:00.149 | DEBUG    | triagerx.model.module_factory:get_model:65 - Including number of filters and classifiers
2024-06-20 13:11:00.150 | DEBUG    | triagerx.model.triagerx_dev_model:__init__:24 - Loading base transformer models: ['microsoft/deberta-base', 'roberta-base']
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [72]:
dev_model.load_state_dict(torch.load(developer_model_weights))

<All keys matched successfully>

In [73]:
comp_model = ModelFactory.get_model(
    model_key="triagerx",
    output_size=6,
    unfrozen_layers=3,
    num_classifiers=3,
    base_models=["microsoft/deberta-base", "roberta-base"],
    dropout=0.2,
    max_tokens=256,
    label_map=comp_id2label,
)

2024-06-20 13:11:05.511 | DEBUG    | triagerx.model.module_factory:get_model:54 - Instantiating model of class: <class 'triagerx.model.triagerx_dev_model.TriagerxDevModel'>
2024-06-20 13:11:05.513 | DEBUG    | triagerx.model.module_factory:get_model:65 - Including number of filters and classifiers
2024-06-20 13:11:05.514 | DEBUG    | triagerx.model.triagerx_dev_model:__init__:24 - Loading base transformer models: ['microsoft/deberta-base', 'roberta-base']
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [74]:
comp_model.load_state_dict(torch.load(component_model_weights))

<All keys matched successfully>

In [75]:
similarity_model = SentenceTransformer("all-mpnet-base-v2")

In [76]:
train_embeddings_path = "/home/mdafifal.mamun/notebooks/triagerX/data/openj9_embeddings/embeddings.npy"

In [77]:
encodings = similarity_model.encode(
    df_train.text.tolist(),
    show_progress_bar=True
)

import numpy as np

np.save(train_embeddings_path, encodings)

Batches:   0%|          | 0/83 [00:00<?, ?it/s]

In [164]:
from datetime import datetime
import json
import os
from collections import defaultdict
from typing import Dict, List, Tuple
import math

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from loguru import logger
from sentence_transformers import util

from triagerx.dataset.text_processor import TextProcessor
from triagerx.model.prediction_model import PredictionModel


class TriagerX:
    """
    A class for recommending developers and components for issues based on prediction models
    and similarity measures.
    """

    def __init__(
        self,
        component_prediction_model: PredictionModel,
        developer_prediction_model: PredictionModel,
        similarity_model: nn.Module,
        train_data: pd.DataFrame,
        train_embeddings: str,
        issues_path: str,
        developer_id_map: Dict[str, int],
        component_id_map: Dict[str, int],        
        device: str,
        similarity_prediction_weight: float, 
        time_decay_factor: float,
        direct_assignment_score: float = 2.5,
        contribution_score: float = 2,
        discussion_score: float = 0.5
    ) -> None:
        """
        Initializes the TriagerX system with models, data, and configurations.

        Args:
            component_prediction_model (PredictionModel): Model to predict components.
            developer_prediction_model (PredictionModel): Model to predict developers.
            similarity_model (nn.Module): Model to compute similarity between issues.
            train_data (pd.DataFrame): Training data containing issue information.
            train_embeddings (str): Path to precomputed embeddings of training issues.
            issues_path (str): Path to the directory containing issue files.
            developer_id_map (Dict[str, int]): Mapping from developer names to IDs.
            component_id_map (Dict[str, int]): Mapping from component names to IDs.
            component_developers_map (Dict[str, List[str]]): Mapping from components to associated developers.
            device (str): Device to run the models on (e.g., 'cpu' or 'cuda').
        """
        self._component_prediction_model = component_prediction_model.to(device)
        self._developer_prediction_model = developer_prediction_model.to(device)
        self._similarity_model = similarity_model
        self._device = device
        self._similarity_prediction_weight = similarity_prediction_weight
        self._time_decay_factor = time_decay_factor
        self._direct_assignment_score = direct_assignment_score
        self._contribution_score = contribution_score
        self._discussion_score = discussion_score
        self._train_data = train_data
        self._issues_path = issues_path
        self._all_issues = os.listdir(issues_path)
        self._developer2id_map = developer_id_map
        self._component2id_map = component_id_map
        self._id2developer_map = {idx: dev for dev, idx in developer_id_map.items()}
        self._id2component_map = {idx: comp for comp, idx in component_id_map.items()}
        # self._component_developers_map = component_developers_map
        self._expected_developers = expected_users
        logger.debug(f"Using device: {device}")
        logger.debug("Loading embeddings for existing issues...")
        self._all_embeddings = np.load(train_embeddings)

    def get_recommendation(
        self, issue: str, k_comp: int, k_dev: int, k_rank: int, similarity_threshold: float
    ) -> Dict[str, List]:
        # logger.debug("Predicting components...")
        comp_prediction_score, predicted_components = self._get_predicted_components(issue, k_comp)
        predicted_components_name = [self._id2component_map[idx] for idx in predicted_components]
        # logger.info(f"Predicted components: {predicted_components_name}")

        # logger.debug("Generating developer recommendations...")
        dev_prediction_score = self._get_recommendation_scores_from_dev_model(issue)
        prediction_score, predicted_devs = dev_prediction_score.topk(k_dev, 0, True, True)

        predicted_developers_name = [self._id2developer_map[idx] for idx in predicted_devs.cpu().numpy()]
        # logger.info(f"Recommended developers: {predicted_developers_name}")

        # logger.debug("Generating recommendations by similarity...")
        dev_predictions_by_similarity = self._get_recommendation_by_similarity(
            issue, predicted_components, k_rank, similarity_threshold
        )

        similarity_devs = [dev[0] for dev in dev_predictions_by_similarity]
        similarity_scores = [score[1] for score in dev_predictions_by_similarity]

        normalized_similarity_score = np.array([])

        if similarity_scores:
            normalized_similarity_score = self.normalize(similarity_scores)

            for i, (sim_dev, sim_score) in enumerate(dev_predictions_by_similarity):            
                dev_id = self._developer2id_map[sim_dev]
                dev_prediction_score[dev_id] += (normalized_similarity_score[i] * self._similarity_prediction_weight)

        
        similar_issue_devs = [dev_sim for dev_sim, _ in dev_predictions_by_similarity]
        # logger.info(f"Recommended developers by issue similarity: {similar_issue_devs}")

        # logger.debug("Aggregating rankings...")
        aggregated_prediction_score, aggregated_predicted_devs = dev_prediction_score.topk(k_dev, 0, True, True)
        aggregated_rank = [self._id2developer_map[idx] for idx in aggregated_predicted_devs.cpu().numpy()]

        # rank_lists = [predicted_developers_name, similar_issue_devs]
        # aggregated_rank = self._aggregate_rank(rank_lists)[:k_dev]
        # logger.info(f"Recommended developers by ranking aggregation: {aggregated_rank}")

        recommendations = {
            # "predicted_components": predicted_components_name,
            # "comp_prediction_score": comp_prediction_score,
            "predicted_developers": predicted_developers_name[:k_dev],
            "dev_prediction_score": prediction_score[:k_dev],
            "similar_devs": similarity_devs[:k_dev],
            "similar_score": normalized_similarity_score.tolist()[:k_dev],
            "combined_ranking": aggregated_rank,
            "combined_ranking_score": aggregated_prediction_score
        }

        return recommendations

    def _aggregate_rank(self, rank_lists: List[List[str]]) -> List[str]:
        borda_scores = defaultdict(int)
        for rank_list in rank_lists:
            for i, item in enumerate(rank_list):
                borda_scores[item] += len(rank_list) - i
        sorted_items = sorted(borda_scores.items(), key=lambda x: x[1], reverse=True)
        return [item[0] for item in sorted_items]

    def normalize(self, scores):
        min_score = np.min(scores)
        max_score = np.max(scores)

        return (scores - min_score) / (max_score - min_score)
    
    def normalize_tensor(self, tensor):
        min_val = torch.min(tensor)
        max_val = torch.max(tensor)
        normalized_tensor = (tensor - min_val) / (max_val - min_val)
        return normalized_tensor

    def _get_recommendation_scores_from_dev_model(self, issue: str) -> torch.Tensor:
        self._developer_prediction_model.eval()
        with torch.no_grad():
            tokenized_issue = self._developer_prediction_model.tokenize_text(issue)
            predictions = self._developer_prediction_model(tokenized_issue)
        
        output = torch.sum(torch.stack(predictions), 0)
        normalized_score = self.normalize_tensor(output.squeeze(dim=0))
        
        return normalized_score

    def _get_predicted_components(self, issue: str, k: int) -> Tuple[List[float], List[int]]:
        self._component_prediction_model.eval()
        with torch.no_grad():
            tokenized_issue = self._component_prediction_model.tokenize_text(issue)
            predictions = self._component_prediction_model(tokenized_issue)
        output = torch.sum(torch.stack(predictions), 0)
        prediction_score, predicted_components = output.topk(k, 1, True, True)
        return (
            F.softmax(prediction_score.squeeze(dim=0), dim=0).cpu().numpy().tolist(),
            predicted_components.squeeze(dim=0).cpu().numpy().tolist()
        )

    def _get_recommendation_by_similarity(
        self, issue: str, predicted_components: List[int], k_issue: int, similarity_threshold: float
    ) -> List[Tuple[str, float]]:
        
        similar_issues = self._get_top_k_similar_issues(issue, k_issue, similarity_threshold)
        historical_contribution = self._get_historical_contributors(similar_issues)
        # logger.debug(historical_contribution)
        
        return historical_contribution
    
    def _get_historical_contributors(
        self, similar_issues: List[Tuple[int, float]]
    ) -> List[Tuple[str, float]]:
        user_contribution_scores = {}
        skipped_users = set()
        for issue_index, sim_score in similar_issues:
            base_points = sim_score
            issue = self._train_data.iloc[issue_index]
            issue_number = issue.issue_number
            contributors = self._get_contribution_data(issue_number)
            for key, users in contributors.items():
                for user_data in users:
                    user = user_data[0]
                    created_at = user_data[1] if len(user_data) > 1 else None
                    
                    if user not in self._expected_developers:
                        skipped_users.add(user)
                        continue

                    contribution_point = self._discussion_score
                    if key in ["pull_request", "commits"]:
                        contribution_point = self._contribution_score
                    elif key in ["last_assignment", "direct_assignment"]:
                        contribution_point = self._direct_assignment_score

                    # Calculate time decay only if created_at is available
                    time_decay = 1
                    if created_at:
                        contribution_date = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%SZ")
                        days_since_contribution = (datetime.now() - contribution_date).days
                        time_decay = math.exp(-self._time_decay_factor * days_since_contribution)

                    user_contribution_scores[user] = (
                        user_contribution_scores.get(user, 0) + base_points * contribution_point * time_decay
                    )

        # if skipped_users:
        #     logger.warning(
        #         f"Skipped users: {skipped_users} because they don't exist in the expected developers list"
        #     )
        return sorted(user_contribution_scores.items(), key=lambda x: x[1], reverse=True)

    def _get_contribution_data(self, issue_number: int) -> Dict[str, List[Tuple[str, str]]]:
        contributions = {}
        issue_file = f"{issue_number}.json"
        last_assignment = None
        
        if issue_file in self._all_issues:
            with open(os.path.join(self._issues_path, issue_file), "r") as file:
                issue = json.load(file)
                assignees = issue.get("assignees", [])
                assignee_logins = [(assignee["login"], None) for assignee in assignees] if assignees else []
                contributions["direct_assignment"] = assignee_logins
                timeline = issue.get("timeline_data", [])
                pull_requests, commits, discussion = [], [], []
                
                for timeline_event in timeline:
                    event = timeline_event.get("event")
                    created_at = timeline_event.get("created_at")
                    if event == "cross-referenced" and timeline_event["source"].get("issue", {}).get("pull_request"):
                        actor = timeline_event["actor"]["login"]
                        pull_requests.append((actor, created_at))
                        last_assignment = actor
                    elif event == "referenced" and timeline_event.get("commit_url"):
                        actor = timeline_event["actor"]["login"]
                        commits.append((actor, created_at))
                        last_assignment = actor
                    elif event == "commented":
                        actor = timeline_event["actor"]["login"]
                        discussion.append((actor, created_at))
                
                contributions["pull_request"] = pull_requests
                contributions["commits"] = commits
                contributions["discussion"] = discussion
                contributions["last_assignment"] = [(last_assignment, None)] if last_assignment else []
        return contributions

    def _get_top_k_similar_issues(self, issue: str, k: int, threshold: float) -> List[Tuple[int, float]]:
        issue_embedding = self._similarity_model.encode(issue)
        cos_sim = util.cos_sim(issue_embedding, self._all_embeddings)
        topk_values, topk_indices = torch.topk(cos_sim, k=k)
        topk_values = topk_values.cpu().numpy()[0]
        topk_indices = topk_indices.cpu().numpy()[0]

        return [
            (idx, sim_score) for idx, sim_score in zip(topk_indices, topk_values) if sim_score >= threshold
        ]


# Grid Search

In [165]:
def get_recommendation(trx, test_idx, k_comp, k_dev, k_rank, sim):
    test_data = df_test.iloc[test_idx]

    # print("Issue URL: ", test_data["issue_url"])
    # print("Actual owner: ", test_data["owner"])
    # print("Actual component: ", test_data["component"])

    return trx.get_recommendation(
        test_data.text,
        k_comp=k_comp,
        k_dev=k_dev,
        k_rank=k_rank,
        similarity_threshold=sim
    )

In [179]:
def get_topk_score(recommendations, top_k):
    combined_total = 0
    dl_total = 0
    sim_total = 0

    for idx in range(len(df_test)):
        actual = df_test.iloc[idx]["owner"]
        combined_recommended = recommendations[idx]["combined_ranking"][:top_k]
        dl_recommended = recommendations[idx]["predicted_developers"][:top_k]
        sim_recommended = recommendations[idx]["similar_devs"][:top_k]

        if actual in combined_recommended:
            combined_total += 1

        if actual in dl_recommended:
            dl_total += 1

        if actual in sim_recommended:
            sim_total += 1

    return dl_total/len(df_test), sim_total/len(df_test), combined_total/len(df_test)
    

In [167]:
def evaluate_recommendations(params):
    # Extract parameters
    similarity_prediction_weight = params['similarity_prediction_weight']
    time_decay_factor = params['time_decay_factor']
    direct_assignment_score = params['direct_assignment_score']
    contribution_score = params['contribution_score']
    discussion_score = params['discussion_score']
    similarity_threshold = params['similarity_threshold']

    trx = TriagerX(
        component_prediction_model=comp_model,
        developer_prediction_model=dev_model,
        similarity_model=similarity_model,
        issues_path="/home/mdafifal.mamun/notebooks/triagerX/data/openj9/openj9_issue_data_6_7_24",
        train_embeddings=train_embeddings_path,
        developer_id_map=lbl2idx,
        component_id_map=comp_lbl2id,
        train_data=df_train,
        device="cuda",
        similarity_prediction_weight=similarity_prediction_weight,
        time_decay_factor=time_decay_factor,
        direct_assignment_score=direct_assignment_score,
        contribution_score=contribution_score,
        discussion_score=discussion_score
    )

    recommendations = []

    for i in tqdm(range(len(df_test)), total=len(df_test), desc="Processing..."):
        rec = get_recommendation(trx, i, k_comp=3, k_dev=20, k_rank=20, sim=similarity_threshold)
        recommendations.append(rec)

    top_3 = get_topk_score(recommendations, 3)
    top_5 = get_topk_score(recommendations, 5)
    top_10 = get_topk_score(recommendations, 10)
    top_20 = get_topk_score(recommendations, 20)

    return top_3, top_5, top_10, top_20

In [181]:
parameter_ranges = {
    'similarity_prediction_weight': [0.6, 0.7, 0.8, 0.9, 1.0],
    'time_decay_factor': [0.01, 0.02, 0.03, 0.04, 0.05],
    'direct_assignment_score': [1.0, 1.5, 2.0, 2.5],
    'contribution_score': [1.0, 1.5, 2.0, 2.5],
    'discussion_score': [0.5, 0.7, 1.0],
    'similarity_threshold': [0.5, 0.6, 0.7]
}

In [182]:
len(list(itertools.product(*parameter_ranges.values())))

3600

In [180]:
import itertools

total_combinations = len(list(itertools.product(*parameter_ranges.values())))

# Initialize an empty list to store results
results = []

index = 1

# Iterate over all combinations
for params in itertools.product(*parameter_ranges.values()):
    
    print(f"Running Grid Search... {index}/{total_combinations}")
    index += 1
    
    params_dict = {
        'similarity_prediction_weight': params[0],
        'time_decay_factor': params[1],
        'direct_assignment_score': params[2],
        'contribution_score': params[3],
        'discussion_score': params[4],
        'similarity_threshold': params[5]
    }

    top_3, top_5, top_10, top_20 = evaluate_recommendations(params_dict)

    # Append results to the list
    results.append({
        'similarity_prediction_weight': params_dict['similarity_prediction_weight'],
        'time_decay_factor': params_dict['time_decay_factor'],
        'direct_assignment_score': params_dict['direct_assignment_score'],
        'contribution_score': params_dict['contribution_score'],
        'discussion_score': params_dict['discussion_score'],
        'similarity_threshold': params_dict['similarity_threshold'],
        'T3DL': top_3[0],
        'T3Sim': top_3[1],
        'T3Com': top_3[2],
        'T5DL': top_5[0],
        'T5Sim': top_5[1],
        'T5Comb': top_5[2],
        'T10DL': top_10[0],
        'T10Sim': top_10[1],
        'T10Com': top_10[2],
        'T20DL': top_20[0],
        'T20Sim': top_20[1],
        'T20Com': top_20[2],
    })

# Convert the results list into a pandas DataFrame
df = pd.DataFrame(results)

# Define output file name
output_file = 'grid_search_results.csv'

# Write DataFrame to CSV file
df.to_csv(output_file, index=False)

print(f"Grid search results saved to {output_file}")

2024-06-20 14:29:05.945 | DEBUG    | __main__:__init__:76 - Using device: cuda
2024-06-20 14:29:05.946 | DEBUG    | __main__:__init__:77 - Loading embeddings for existing issues...


Running Grid Search... 1/2


Processing...:  59%|█████▉    | 173/293 [00:21<00:16,  7.33it/s]/tmp/ipykernel_525931/3636110859.py:149: RuntimeWarning: invalid value encountered in divide
  return (scores - min_score) / (max_score - min_score)
Processing...: 100%|██████████| 293/293 [00:36<00:00,  8.00it/s]
2024-06-20 14:29:42.653 | DEBUG    | __main__:__init__:76 - Using device: cuda
2024-06-20 14:29:42.654 | DEBUG    | __main__:__init__:77 - Loading embeddings for existing issues...


Running Grid Search... 2/2


Processing...: 100%|██████████| 293/293 [00:37<00:00,  7.89it/s]


Grid search results saved to grid_search_results.csv


In [134]:
ff = get_recommendation(27, k_comp=3, k_dev=3, k_rank=20, sim=0.7)

2024-06-20 13:51:52.430 | WARNING  | __main__:_get_historical_contributors:236 - Skipped users: {'sophia-guo', 'DanHeidinga', 'Mesbah-Alam', 'dipak-bagadiya', 'mikezhang1234567890', 'karianna'} because they don't exist in the expected developers list


Issue URL:  https://github.com/eclipse-openj9/openj9/issues/17871
Actual owner:  babsingh
Actual component:  comp:vm


In [137]:
recommendations[1]

{'predicted_developers': ['JasonFengJ9',
  'babsingh',
  'llxia',
  'hzongaro',
  'a7ehuo',
  'r30shah',
  'hangshao0',
  'mpirvu',
  'gacholio',
  'ChengJin01',
  'jdmpapin',
  'thallium',
  'dsouzai',
  'smlambert',
  'nbhuiyan',
  'BradleyWood',
  'keithc-ca',
  'pshipton',
  'EricYangIBM',
  'LongyuZhang'],
 'dev_prediction_score': tensor([1.0000, 0.9983, 0.9162, 0.9072, 0.8675, 0.8621, 0.8562, 0.8331, 0.7641,
         0.7459, 0.7377, 0.7168, 0.6989, 0.6806, 0.6745, 0.6640, 0.6605, 0.6590,
         0.6511, 0.6480], device='cuda:0'),
 'similar_devs': ['pshipton',
  'knn-k',
  'mpirvu',
  'keithc-ca',
  'llxia',
  'hangshao0',
  'hzongaro',
  'Akira1Saitoh',
  'BradleyWood',
  'cjjdespres',
  'a7ehuo',
  'r30shah',
  'jdmpapin',
  '0xdaryl',
  'JasonFengJ9',
  'tajila',
  'gacholio',
  'gita-omr',
  'babsingh'],
 'similar_score': [1.0,
  0.8239102707271186,
  0.5334083997652541,
  0.5045742298497052,
  0.49946390672608015,
  0.28976064086595715,
  0.28222617310320464,
  0.27550697777

In [144]:
recommendations = []

for i in range(len(df_test)):
    rec = get_recommendation(i, k_comp=3, k_dev=20, k_rank=20, sim=0.6)
    recommendations.append(rec)

Issue URL:  https://github.com/eclipse-openj9/openj9/issues/17734
Actual owner:  a7ehuo
Actual component:  comp:jit


2024-06-20 13:55:59.925 | WARNING  | __main__:_get_historical_contributors:236 - Skipped users: {'liqunl', 'DanHeidinga', 'M-Davies', 'andrewcraik', 'R-Santhir', 'andrew-m-leonard', 'manashaVetrivelu', 'dchopra001', 'zl-wang', 'cathyzhyi', 'pdbain-ibm', 'mikezhang1234567890', 'irinarada', 'lumpfish', 'fjeremic', 'IBMJimmyk', 'dylanjtuttle'} because they don't exist in the expected developers list
2024-06-20 13:56:00.103 | WARNING  | __main__:_get_historical_contributors:236 - Skipped users: {'manashaVetrivelu', 'cathyzhyi', 'irinarada', 'Talk2Harish', 'dylanjtuttle', 'DanHeidinga', 'dmitry-ten', 'R-Santhir', 'AlenBadel', 'andrew-m-leonard', 'dchopra001', 'pdbain-ibm', 'OussamaSaoudi', 'harryyu1994', 'fjeremic', 'Leonardo2718', 'liqunl', 'jluehe', 'andrewcraik', 'mikezhang1234567890'} because they don't exist in the expected developers list


Issue URL:  https://github.com/eclipse-openj9/openj9/issues/17736
Actual owner:  dsouzai
Actual component:  comp:jit
Issue URL:  https://github.com/eclipse-openj9/openj9/issues/17738
Actual owner:  JasonFengJ9
Actual component:  comp:vm


2024-06-20 13:56:00.255 | WARNING  | __main__:_get_historical_contributors:236 - Skipped users: {'DanHeidinga', 'dmitry-ten', 'mateuszrzeszutek', 'vij-singh', 'Spencer-Comin', 'dchopra001', 'EmanElsaban', 'mikezhang1234567890', 'manqingl'} because they don't exist in the expected developers list
2024-06-20 13:56:00.373 | WARNING  | __main__:_get_historical_contributors:236 - Skipped users: {'liqunl', 'DanHeidinga', 'dmitry-ten', 'XxAdi101xX', 'dchopra001', 'harryyu1994', 'ashu-mehra'} because they don't exist in the expected developers list


Issue URL:  https://github.com/eclipse-openj9/openj9/issues/17748
Actual owner:  dsouzai
Actual component:  comp:test
Issue URL:  https://github.com/eclipse-openj9/openj9/issues/17757
Actual owner:  keithc-ca
Actual component:  comp:build


...

2024-06-20 13:56:36.553 | WARNING  | __main__:_get_historical_contributors:236 - Skipped users: {'StephanieMeakins', 'liqunl', 'groeges', 'DanHeidinga', 'dmitry-ten', 'andrewcraik', 'Mesbah-Alam', 'AidanHa', 'patkarns', 'pdbain-ibm', 'manashaVetrivelu', 'harryyu1994', 'mikezhang1234567890', 'ben-walsh', 'fjeremic', 'pdbain', 'VermaSh', 'dylanjtuttle'} because they don't exist in the expected developers list
2024-06-20 13:56:36.636 | WARNING  | __main__:_get_historical_contributors:236 - Skipped users: {'singh264'} because they don't exist in the expected developers list


Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19534
Actual owner:  JasonFengJ9
Actual component:  comp:vm
Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19543
Actual owner:  pshipton
Actual component:  comp:vm
Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19546
Actual owner:  nbhuiyan
Actual component:  comp:jit


2024-06-20 13:56:36.838 | WARNING  | __main__:_get_historical_contributors:236 - Skipped users: {'dmitry-ten', 'Mesbah-Alam', 'vij-singh', 'dchopra001', 'zl-wang', 'vijaysun-omr', 'dnakamura', 'ninja-', 'fjeremic', 'harryyu1994', 'ashu-mehra'} because they don't exist in the expected developers list
2024-06-20 13:56:36.946 | WARNING  | __main__:_get_historical_contributors:236 - Skipped users: {'ThanHenderson', 'liqunl', 'DanHeidinga', 'yanluo7', 'jluehe', 'vij-singh', 'andrew-m-leonard', 'pdbain-ibm', 'zl-wang', 'AlenBadel', 'irinarada', 'midronij', 'taoliult', 'Talk2Harish', 'mnalam-p', 'groeges'} because they don't exist in the expected developers list


Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19583
Actual owner:  JasonFengJ9
Actual component:  comp:vm
Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19605
Actual owner:  pshipton
Actual component:  comp:vm


2024-06-20 13:56:37.065 | WARNING  | __main__:_get_historical_contributors:236 - Skipped users: {'DanHeidinga', 'fuzzy000', 'dmitry-ten', 'yanluo7', 'acrowthe', 'pdbain-ibm', 'AlenBadel', 'psoujany', 'adamfarley', 'sharon-wang', 'mikezhang1234567890', 'ben-walsh', 'harryyu1994', 'dholmes-ora'} because they don't exist in the expected developers list


In [151]:
combined_total = 0
dl_total = 0
sim_total = 0
top_k = 10

for idx in range(len(df_test)):
    actual = df_test.iloc[idx]["owner"]
    combined_recommended = recommendations[idx]["combined_ranking"][:top_k]
    dl_recommended = recommendations[idx]["predicted_developers"][:top_k]
    sim_recommended = recommendations[idx]["similar_devs"][:top_k]

    # print(actual)
    # print(sim_recommended)

    # if sim_recommended:
    #     sim_recommended = [s[0] for s in sim_recommended]

    print(actual in sim_recommended)

    print(idx, end=" ")

    if actual in combined_recommended:
        print("Comb,", end="")
        combined_total += 1

    if actual in dl_recommended:
        print("DL,", end="")
        dl_total += 1

    if actual in sim_recommended:
        print("Sim", end="")
        sim_total += 1

    print("===")

True
0 Comb,DL,Sim===
False
1 ===
True
2 Comb,Sim===
True
3 Comb,DL,Sim===
True
4 Comb,DL,Sim===
True
5 Comb,DL,Sim===
True
6 Sim===
True
7 Comb,DL,Sim===
True
8 Comb,DL,Sim===
True
9 Comb,DL,Sim===
False
10 ===
True
11 Comb,DL,Sim===
True
12 Comb,DL,Sim===
True
13 Comb,DL,Sim===
True
14 Comb,DL,Sim===
False
15 ===
True
16 Comb,DL,Sim===
True
17 Comb,DL,Sim===
False
18 ===
True
19 Comb,DL,Sim===
False
20 ===
False
21 Comb,DL,===
True
22 Comb,DL,Sim===
True
23 Comb,DL,Sim===
True
24 Comb,DL,Sim===
True
25 Comb,DL,Sim===
True
26 Comb,DL,Sim===
True
27 Comb,DL,Sim===
True
28 Comb,DL,Sim===
True
29 Comb,DL,Sim===
False
30 Comb,DL,===
True
31 Comb,DL,Sim===
True
32 Comb,DL,Sim===
True
33 Sim===
False
34 ===
True
35 Comb,DL,Sim===
True
36 Comb,DL,Sim===
True
37 Comb,DL,Sim===
False
38 ===
True
39 Comb,DL,Sim===
True
40 Sim===
False
41 DL,===
False
42 Comb,DL,===
False
43 Comb,DL,===
True
44 Comb,DL,Sim===
False
45 Comb,DL,===
True
46 Comb,DL,Sim===
True
47 Comb,DL,Sim===
False
48 ===
False
4

In [152]:
dl_total/len(df_test), sim_total/len(df_test), combined_total/len(df_test)

(0.7201365187713311, 0.689419795221843, 0.7406143344709898)

In [57]:
get_recommendation(264, k_comp=3, k_dev=3, k_rank=20, sim=0.6)

2024-06-20 12:19:35.239 | DEBUG    | __main__:get_recommendation:87 - Predicting components...
2024-06-20 12:19:35.275 | INFO     | __main__:get_recommendation:90 - Predicted components: ['comp:vm', 'comp:gc', 'comp:test']
2024-06-20 12:19:35.276 | DEBUG    | __main__:get_recommendation:92 - Generating developer recommendations...


Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19334
Actual owner:  pshipton
Actual component:  comp:vm


2024-06-20 12:19:35.311 | INFO     | __main__:get_recommendation:99 - Recommended developers: ['mikezhang1234567890', 'keithc-ca', 'hzongaro']
2024-06-20 12:19:35.312 | DEBUG    | __main__:get_recommendation:101 - Generating recommendations by similarity...
2024-06-20 12:19:35.340 | WARNING  | __main__:_get_historical_contributors:308 - Skipped users: {'dchopra001', 'mikezhang1234567890', 'andrewcraik', 'PascalSchumacher', 'fjeremic', 'DanHeidinga'} because they don't exist in the expected developers list
2024-06-20 12:19:35.341 | DEBUG    | __main__:_get_recommendation_by_similarity:245 - [('pshipton', 4.682542394465974), ('Akira1Saitoh', 1.7882484696126846), ('hzongaro', 1.5787483303407723), ('tajila', 0.0005607664597205145), ('dmitripivkine', 2.73656584099672e-08)]
2024-06-20 12:19:35.343 | INFO     | __main__:get_recommendation:125 - Recommended developers by issue similarity: ['pshipton', 'Akira1Saitoh', 'hzongaro', 'tajila', 'dmitripivkine']
2024-06-20 12:19:35.344 | DEBUG    | _

Aggregated score tensor([1.3422, 1.1808, 1.0000], device='cuda:0')


{'predicted_components': ['comp:vm', 'comp:gc', 'comp:test'],
 'comp_prediction_score': [0.9986054301261902,
  0.0013944254023954272,
  1.2235675228566834e-07],
 'predicted_developers': ['mikezhang1234567890', 'keithc-ca', 'hzongaro'],
 'dev_prediction_score': tensor([1.3422, 1.1808, 1.0000], device='cuda:0'),
 'similar_devs': [('pshipton', 4.682542394465974),
  ('Akira1Saitoh', 1.7882484696126846),
  ('hzongaro', 1.5787483303407723)],
 'combined_ranking': ['pshipton', 'hzongaro', 'mikezhang1234567890']}

In [33]:
anal_id = 47

print(df_test.iloc[anal_id]["issue_url"])
df_test.iloc[anal_id], recommendations[anal_id]

https://github.com/eclipse-openj9/openj9/issues/17962


(Unnamed: 0                                                   7258
 issue_number                                                17962
 issue_title     testDDRExtJunit_FindExtThread Failed to constr...
 description     testDDRExtJunit_FindExtThread_0\r\n```\r\n    ...
 issue_url       https://github.com/eclipse-openj9/openj9/issue...
 issue_state                                                  open
 creator                                                  pshipton
 labels                                      comp:vm, test failure
 owner                                                    pshipton
 component                                                 comp:vm
 text            Bug Title: testDDRExtJunit_FindExtThread Faile...
 owner_id                                                       41
 Name: 59, dtype: object,
 {'predicted_components': ['comp:vm', 'comp:test', 'comp:gc'],
  'comp_prediction_score': [0.8749904036521912,
   0.12500955164432526,
   1.1886748296774385e-07],
  'pred